In [ ]:
"""
Created on Mon May 02 11:15 2022

Correlation matrix input data

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import trange, tqdm
import seaborn as sns

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
inputpath_data = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'
outputpath_nn_models = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/'
outputpath_doc = '/bettik/burgardc/SCRIPTS/basal_melt_neural_networks/custom_doc/'
plot_path = '/bettik/burgardc/PLOTS/NN_plots/input_vars/'

In [ ]:
sorted_isf_all = [11,69,43,12,70,44,58,71,45,30,31,61,73,32,49,18,10,65,51,22,23,66,39,40,75,25,26,42,55]
#sorted_isf_all = [32,33,17,49,18,10,65,51,22,23,66,39,40,75,25,26,42,55]

In [ ]:
clean_df_kisf = pd.read_csv(outputpath_nn + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+str(tt)+'_'+nemo_run+'.csv')

In [ ]:
clean_df_kisf

In [ ]:
for nemo_run in ['bf663']: #'bf663'
    
    whole_input_df = None
    tblock_list = []
    isf_list = []
    for tt in tqdm(range(1980,1980 + 60)):
        
        outputpath_nn = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/SMITH_'+nemo_run+'_EXTRAPDRAFT_CHUNKS/'

        for kisf in sorted_isf_all:
            clean_df_kisf = pd.read_csv(outputpath_nn + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+str(tt)+'_'+nemo_run+'.csv',index_col=[0,1])
            clean_df_kisf.reset_index(drop=True, inplace=True)
            if 'Nisf' in clean_df_kisf.columns.values.tolist():
                clean_df_kisf = clean_df_kisf.drop(['Nisf'], axis=1)
            clean_ds_kisf = clean_df_kisf.to_xarray()
            

            if whole_input_df is None:
                whole_input_df = clean_ds_kisf.copy()
                tblock_list = tblock_list + (np.zeros(len(whole_input_df.index)) + tt).astype(int).tolist()
                isf_list = isf_list + (np.zeros(len(whole_input_df.index)) + kisf).astype(int).tolist()
            else:
                new_index = clean_ds_kisf.index.values + whole_input_df.index.max().values+1
                clean_ds_kisf = clean_ds_kisf.assign_coords({'index': new_index})
                whole_input_df = xr.concat([whole_input_df, clean_ds_kisf], dim='index')
                tblock_list = tblock_list + (np.zeros(len(new_index)) + tt).astype(int).tolist()
                isf_list = isf_list + (np.zeros(len(new_index)) + kisf).astype(int).tolist()

    whole_input_df.to_netcdf(outputpath_nn + 'dataframe_allisf_alltimes.nc')
    index_ds = xr.Dataset({'Nisf': (['index'], isf_list), 'tblock': (['index'], tblock_list)}, coords={'index': whole_input_df.index})
    index_ds.to_netcdf(outputpath_nn + 'indexing_allisf_alltimes.nc')

In [ ]:
kisf

In [ ]:
col_list = ['dGL','dIF','corrected_isfdraft','bathy_metry','slope_bed_lon','slope_bed_lat',
            'slope_ice_lon','slope_ice_lat','theta_in','salinity_in','T_mean','T_std','S_mean','S_std',
                                            'melt_m_ice_per_y']

In [ ]:
nemo_run = 'bf663'
outputpath_nn = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/SMITH_'+nemo_run+'_EXTRAPDRAFT_CHUNKS/'

whole_input_df = xr.open_dataset(outputpath_nn + 'dataframe_allisf_alltimes.nc')

In [ ]:
df_all = whole_input_df.to_dataframe()

In [ ]:
corr_matrix = df_all[col_list].astype(float).corr()

In [ ]:
corr_matrix_round = corr_matrix.round(1)

In [ ]:
corr_matrix_round

In [ ]:
f = plt.figure(figsize=(8.24,8.24))
sns.heatmap(corr_matrix_round, annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')
f.savefig(plot_path+'cross_correlation_input_'+nemo_run+'.png')

In [ ]:
TS_input


In [ ]:
input_dataset